## DeepFillv1 experiment

In [2]:
import cv2
import numpy as np
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # will suppress all warnings
import neuralgym as ng

from deepfillv1.inpaint_model import InpaintCAModel

from utils import Namespace
from utils import get_colormap_image

In [5]:
w, h = 170, 128
w_resize, h_resize = w * 4, h * 4

sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = True
with tf.Session(config=sess_config) as sess:
    colormap = get_colormap_image(w, h, w_resize, h_resize)
    colormap = (colormap + 1.) * 127.5
    colormap = tf.reverse(colormap, [-1])
    colormap = tf.saturate_cast(colormap, tf.uint8)
    colormap = sess.run(colormap)
    colormap = np.array(colormap)
    cv2.imwrite('colorpalette.jpg', colormap[0][:, :, ::-1])

In [7]:
args = Namespace(image = f'deepfillv1/examples/places2/sunset_input.png',
                 mask = f'deepfillv1/examples/places2/sunset_mask.png',
                 flow = f'./flow.png',
                 output = f'./output.png',
                 output_coarse = f'./output_coarse.png',
                 output_fine = f'./output_fine.png',
                 output_flow = f'./output_flow.png',
                 output_modulated = f'./output_modulated.png',
                 checkpoint_dir = f'deepfillv1/model_logs')

ng.get_gpus(1)

    0          -     -     -     -     -     -   -              
    1          -     -     -     -     -     -   -              
    2          -     -     -     -     -     -   -              
    3          -     -     -     -     -     -   -              
    4          -     -     -     -     -     -   -              
    5          -     -     -     -     -     -   -              
    6          -     -     -     -     -     -   -              
    7          -     -     -     -     -     -   -              

Set env: CUDA_VISIBLE_DEVICES=[2].


[2]

# Attention output test

In [8]:
model = InpaintCAModel()
image = cv2.imread(args.image)
mask = cv2.imread(args.mask)

assert image.shape == mask.shape

h, w, _ = image.shape
grid = 8
image = image[:h//grid*grid, :w//grid*grid, :]
mask = mask[:h//grid*grid, :w//grid*grid, :]
print('Shape of image: {}'.format(image.shape))

image = np.expand_dims(image, 0)
mask = np.expand_dims(mask, 0)
input_image = np.concatenate([image, mask], axis=2)

print('Shape of model input: {}'.format(input_image.shape))

Shape of image: (512, 680, 3)
Shape of model input: (1, 512, 1360, 3)


In [4]:
sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = True
with tf.Session(config=sess_config) as sess:
    input_image = tf.constant(input_image, dtype=tf.float32)
    output = model.build_server_graph(input_image)
    output = (output + 1.) * 127.5
    output = tf.reverse(output, [-1])
    output = tf.saturate_cast(output, tf.uint8)
    # load pretrained model
    vars_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    assign_ops = []
    for var in vars_list:
        vname = var.name
        from_name = vname
        var_value = tf.contrib.framework.load_variable(args.checkpoint_dir, from_name)
        assign_ops.append(tf.assign(var, var_value))
    sess.run(assign_ops)
    print('Model loaded.')
    result = sess.run(output)
    print('Shape of model output: {}'.format(result.shape))
    
    result = np.array(result)
    result, coarse, fine, flow = np.split(result, 4, axis=2)
    cv2.imwrite(args.output, result[0][:, :, ::-1])
    cv2.imwrite(args.output_coarse, coarse[0][:, :, ::-1])
    cv2.imwrite(args.output_fine, fine[0][:, :, ::-1])
    cv2.imwrite(args.output_flow, flow[0][:, :, ::-1])
    
print('Done')

Called: build_server_graph()
Shape of masked images: (1, 512, 680, 3)
Shape of masks: (1, 512, 680, 1)
Called: build_inpaint_net()
Shape of first-stage input: (1, 512, 680, 5)
Shape of first-stage output: (1, 512, 680, 3)
Shape of second-stage input: (1, 512, 680, 3)
Shape of second-stage conv branch output: (1, 128, 170, 128)
Shape of contextual attention input: (TensorShape([Dimension(1), Dimension(128), Dimension(170), Dimension(128)]), TensorShape([Dimension(1), Dimension(128), Dimension(170), Dimension(1)]))
Called: contextual_attention()
Shape of foreground features: (1, 128, 170, 128)
Shape of background features: (1, 128, 170, 128)
Shape of masks: (1, 128, 170, 1)
Shape of background patches: (1, 4, 4, 128, 5440)
Shape of resized foreground features: (1, 64, 85, 128)
Shape of resized background features: (1, 64, 85, 128)
Shape of resized masks: (1, 64, 85, 1)
Shape of downscaled background patches (w): (1, 3, 3, 128, 5440)
Shape of mask patches: (1, 1, 1, 5440)
[1/1]: Shapes fg

# Attention input test

In [3]:
model = InpaintCAModel()
image = cv2.imread(args.image)
mask = cv2.imread(args.mask)
flow = cv2.imread(args.flow)

assert image.shape == mask.shape == flow.shape

h, w, _ = image.shape
grid = 8
image = image[:h//grid*grid, :w//grid*grid, :]
mask = mask[:h//grid*grid, :w//grid*grid, :]
print('Shape of image: {}'.format(image.shape))

image = np.expand_dims(image, 0)
mask = np.expand_dims(mask, 0)
input_image = np.concatenate([image, mask], axis=2)
print('Shape of model input: {}'.format(input_image.shape))

flow = flow[:h//grid*grid, :w//grid*grid, :]
input_flow = np.expand_dims(flow, 0)
print('Shape of flow input: {}'.format(input_flow.shape))

Shape of image: (512, 680, 3)
Shape of model input: (1, 512, 1360, 3)
Shape of flow input: (1, 512, 680, 3)


In [4]:
sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = True
with tf.Session(config=sess_config) as sess:
    input_image = tf.constant(input_image, dtype=tf.float32)
    output = model.build_server_graph(input_image, input_flow)
    output = (output + 1.) * 127.5
    output = tf.reverse(output, [-1])
    output = tf.saturate_cast(output, tf.uint8)
    # load pretrained model
    vars_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    assign_ops = []
    for var in vars_list:
        vname = var.name
        from_name = vname
        var_value = tf.contrib.framework.load_variable(args.checkpoint_dir, from_name)
        assign_ops.append(tf.assign(var, var_value))
    sess.run(assign_ops)
    print('Model loaded.')
    result = sess.run(output)
    print('Shape of model output: {}'.format(result.shape))
    
    result = np.array(result)
    result, coarse, fine, flow = np.split(result, 4, axis=2)
    cv2.imwrite(args.output_modulated, result[0][:, :, ::-1])
    
print('Done')

Called: build_server_graph()
Shape of masked images: (1, 512, 680, 3)
Shape of masks: (1, 512, 680, 1)
Called: build_inpaint_net()
Shape of first-stage input: (1, 512, 680, 5)
Shape of first-stage output: (1, 512, 680, 3)
Shape of second-stage input: (1, 512, 680, 3)
Shape of second-stage conv branch output: (1, 128, 170, 128)
Shape of contextual attention input: (TensorShape([Dimension(1), Dimension(128), Dimension(170), Dimension(128)]), TensorShape([Dimension(1), Dimension(128), Dimension(170), Dimension(1)]))
Called: contextual_attention()
Shape of foreground features: (1, 128, 170, 128)
Shape of background features: (1, 128, 170, 128)
Shape of masks: (1, 128, 170, 1)
Shape of background patches: (1, 4, 4, 128, 5440)
Shape of resized foreground features: (1, 64, 85, 128)
Shape of resized background features: (1, 64, 85, 128)
Shape of resized masks: (1, 64, 85, 1)
Shape of downscaled background patches (w): (1, 3, 3, 128, 5440)
Shape of mask patches: (1, 1, 1, 5440)
[1/1]: Shapes fg